In [2]:
import requests
import pandas as pd
from typing import Text
from flask import Flask, render_template
import pandas as pd
import requests
from datetime import timedelta, datetime

In [6]:
thisGw = 30
def getCap(playerId):
    url2 = 'https://fantasy.premierleague.com/api/entry/' + str(playerId)+ '/event/' + str  (thisGw) + '/picks/'
    r2 = requests.get(url2)
    picks = r2.json()['picks']

    for pick in picks:
        if pick['is_captain']:
            return pick['element']

In [7]:
getCap(3378570)

388

In [9]:

url4 = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r4 = requests.get(url4)
json = r4.json()['teams']

In [11]:
json[0]

{'code': 3,
 'draw': 0,
 'form': None,
 'id': 1,
 'loss': 0,
 'name': 'Arsenal',
 'played': 0,
 'points': 0,
 'position': 0,
 'short_name': 'ARS',
 'strength': 4,
 'team_division': None,
 'unavailable': False,
 'win': 0,
 'strength_overall_home': 1190,
 'strength_overall_away': 1210,
 'strength_attack_home': 1170,
 'strength_attack_away': 1210,
 'strength_defence_home': 1190,
 'strength_defence_away': 1200,
 'pulse_id': 1}